In [1]:
import sys
sys.path.append('..')
from config_notebooks import *
from qdrant.qdrant_helper import *
from azureopenaimanager.azureopenai_helper import AzureOpenAIManager
from azure_blob.azure_blob_helper import AzureBlobHelper
from sentence_transformers import CrossEncoder

c:\Ambarish\text2sql\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
search = QdrantHelper(QDRANT_URL,QDRANT_KEY,MODEL_NAME,QDRANT_COLLECTION,
                      qmodels.Distance.COSINE, 384,25)

In [3]:
query = """
What is De minimis non curat lex ? 
"""

In [4]:
CATEGORY = ""
USER_ID = "ambarish.ganguly@gmail.com"

In [5]:
results_to_return, \
metadata_source_filename_to_return, \
metadata_source_page_to_return, \
reranker_score_to_return = search.get_search_results(user_input=query,
                                                     CATEGORY=CATEGORY,
                                                     user_id=USER_ID,
                                                     )


What is De minimis non curat lex ? 

################################
[-2.08998844e-02  4.91481349e-02 -2.74850558e-02  5.30819548e-03
 -1.93886217e-02  2.41186302e-02  2.97604147e-02  1.27572700e-01
 -1.01857008e-02  5.34148421e-03  1.01855814e-01 -3.54509093e-02
 -5.75582683e-02 -4.28360328e-02 -6.24046437e-02 -4.02064137e-02
  7.76625797e-02  2.77915671e-02  1.38834855e-02  2.18491908e-02
 -1.31272990e-02 -1.46306595e-02 -3.25174630e-02  9.00009181e-03
 -3.25461030e-02  8.96731317e-02 -5.42254113e-02  3.75776067e-02
  1.11698695e-01 -7.37310201e-02  1.03579648e-02  2.91921366e-02
  5.76902553e-02  3.11545562e-02  4.81735952e-02 -4.64341007e-02
 -1.54733723e-02  2.27608141e-02  3.15641649e-02 -4.47567217e-02
 -3.76142450e-02 -8.28191359e-03 -9.66505185e-02  2.36762259e-02
 -4.38305177e-02 -5.30715473e-02 -3.53014395e-02  1.56562980e-02
 -5.12038767e-02 -5.38477153e-02  4.97146249e-02  5.39638009e-03
  1.63749848e-02  3.00530344e-02  1.70922966e-03 -4.52267826e-02
 -6.11952208e-02 -3

In [6]:
metadata_source_filename_to_return

['Unit-12.pdf',
 'Unit-12.pdf',
 'air qual 12.pdf',
 'Unit-12.pdf',
 'Unit-12.pdf',
 'air qual 12.pdf',
 'Unit-12.pdf',
 'air qual 12.pdf',
 'Unit-12.pdf',
 'air qual 12.pdf',
 'air qual 12.pdf',
 'air qual 12.pdf',
 'air qual 12.pdf',
 'air qual 12.pdf',
 'CHAP02-BIOLOGY-CLASS11.pdf',
 'Unit-12.pdf',
 'Unit-12.pdf',
 'Unit-12.pdf',
 'air qual 12.pdf',
 'Unit-12.pdf',
 'air qual 12.pdf',
 'air qual 12.pdf',
 'CHAP02-BIOLOGY-CLASS11.pdf',
 'air qual 12.pdf',
 'air qual 12.pdf']

In [7]:
metadata_source_page_to_return

['Unit-12-8.pdf',
 'Unit-12-10.pdf',
 'air qual 12-3.pdf',
 'Unit-12-5.pdf',
 'Unit-12-8.pdf',
 'air qual 12-3.pdf',
 'Unit-12-7.pdf',
 'air qual 12-3.pdf',
 'Unit-12-5.pdf',
 'air qual 12-8.pdf',
 'air qual 12-4.pdf',
 'air qual 12-8.pdf',
 'air qual 12-7.pdf',
 'air qual 12-6.pdf',
 'CHAP02-BIOLOGY-CLASS11-5.pdf',
 'Unit-12-7.pdf',
 'Unit-12-4.pdf',
 'Unit-12-4.pdf',
 'air qual 12-1.pdf',
 'Unit-12-0.pdf',
 'air qual 12-1.pdf',
 'air qual 12-5.pdf',
 'CHAP02-BIOLOGY-CLASS11-4.pdf',
 'air qual 12-8.pdf',
 'air qual 12-9.pdf']

In [8]:
reranker_score_to_return

[]

In [ ]:
results_to_return[0]

In [ ]:
#We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [ ]:
for i in range(len(results_to_return)):
               print(results_to_return[i])

In [ ]:
# Now, score all retrieved passages with the cross_encoder
cross_inp = [[query, results_to_return[i]] for i in range(len(results_to_return))]
cross_scores = cross_encoder.predict(cross_inp)

In [ ]:
cross_scores

In [ ]:
def get_reply(user_input, content):

    azure_open_ai_manager = AzureOpenAIManager(
                    endpoint=AZURE_OPENAI_ENDPOINT,
                    api_key=AZURE_OPENAI_KEY,
                    deployment_id=AZURE_OPENAI_DEPLOYMENT_ID,
                    api_version="2023-05-15"
                )
    
    conversation=[{"role": "system", "content": "If the answer is not found within the context, please mention \
        that the answer is not found \
        Do not answer anything which is not in the context"}]
    reply = azure_open_ai_manager.generate_reply_from_context(user_input, content, conversation)
    return reply

In [ ]:
azure_blob_helper = AzureBlobHelper(AZ_ST_ACC_NAME,
                                    AZ_ST_ACC_KEY,
                                    AZ_ST_CONTAINER_NAME)

In [ ]:
context = "\n".join(results_to_return)
reply = get_reply(query, context)

URLs = []

reranker_confidence = ""

for page in metadata_source_page_to_return:
    URLs.append(azure_blob_helper.generate_sas_url(page))


In [ ]:
reply